In [ ]:
%%capture
!pip install coordination_network_toolkit
!pip install networkx

In [ ]:
import coordination_network_toolkit as coord_net_tk
import networkx as nx
import pandas as pd
import numpy as np
import time
from datetime import *
from dateutil import parser
import re

from math import pow
from string import ascii_letters, digits

# RFC 2396 unreserved URI characters
characters = ascii_letters + digits
size = len(characters)
seq = range(0,size)

dictionary = dict(zip(seq, characters))
reverse_dictionary = dict((v,k) for k,v in dictionary.items())

def encode(n):
  if n == "" :
    chars = ""
  else:
    d = []
    n = n
    while n > 0:
        qr = divmod(n, size)
        n = qr[0]
        d.append(qr[1])
    chars = ''
    for i in d:
        chars += dictionary[i]
    return chars

def convert_date_to_float(text):
  text = parser.parse(text)
  #text = re.sub('[^0-9]','', str(text)) #to integer
  text = text.timestamp()
  return text

In [ ]:
%%capture

##START WITH APRIL WEEKLY DATA
!wget https://www.dropbox.com/s/z2m4hi2r1varu0j/hydrate-clean_dataset_522_clean.tsv
!wget https://www.dropbox.com/s/sti0m1q9up3doak/hydrate-clean_dataset_523_clean.tsv
!wget https://www.dropbox.com/s/fmpj89x08g1p1m7/hydrate-clean_dataset_524_clean.tsv
!wget https://www.dropbox.com/s/i2nmew5gewcceje/hydrate-clean_dataset_525_clean.tsv
!wget https://www.dropbox.com/s/fw6e1yic9u26neo/hydrate-clean_dataset_526_clean.tsv
!wget https://www.dropbox.com/s/l7842grp16x065d/hydrate-clean_dataset_527_clean.tsv
!wget https://www.dropbox.com/s/lri8e8dbz398fuc/hydrate-clean_dataset_528_clean.tsv
!wget https://www.dropbox.com/s/ft2rb2397r9w5yr/hydrate-clean_dataset_529_clean.tsv
!wget https://www.dropbox.com/s/nummf1hi4i1j6be/hydrate-clean_dataset_530_clean.tsv
!wget https://www.dropbox.com/s/4mhsowqf5li4vj2/hydrate-clean_dataset_531_clean.tsv

In [ ]:
column_names = ["timestamp","message_id","tweet_id_str","message","tweet_truncated","tweet_source","reply_id","tweet_in_reply_to_status_id_str","tweet_in_reply_to_user_id","tweet_in_reply_to_user_id_str","tweet_in_reply_to_screen_name","tweet_geo","tweet_coordinates","tweet_place","tweet_contributors","user_id","tweet_user_id_str","tweet_user_name","username","tweet_user_location","tweet_user_description","tweet_place_id","tweet_place_url","tweet_place_place_type","tweet_place_name","tweet_place_full_name","tweet_place_country_code","tweet_place_country","tweet_place_contained_within","tweet_place_bounding_box_type","tweet_place_bounding_box_coordinates","tweet_geo_type", "tweet_geo_coordinates","tweet_coordinates_type","tweet_coordinates_coordinates","lang","retweet_count","favorite_count","urls"]

tsv_read1 = pd.read_csv("hydrate-clean_dataset_522_clean.tsv", sep='\t')
tsv_read1.columns = column_names
tsv_read1 = tsv_read1[tsv_read1.lang=="en"]

tsv_read2 = pd.read_csv("hydrate-clean_dataset_523_clean.tsv", sep='\t')
tsv_read2.columns = column_names
tsv_read2 = tsv_read2[tsv_read2.lang=="en"]


tsv_read3 = pd.read_csv("hydrate-clean_dataset_524_clean.tsv", sep='\t')
tsv_read3.columns = column_names
tsv_read3 = tsv_read3[tsv_read3.lang=="en"]


tsv_read4 = pd.read_csv("hydrate-clean_dataset_525_clean.tsv", sep='\t')
tsv_read4.columns = column_names
tsv_read4 = tsv_read4[tsv_read4.lang=="en"]


tsv_read5 = pd.read_csv("hydrate-clean_dataset_526_clean.tsv", sep='\t')
tsv_read5.columns = column_names
tsv_read5 = tsv_read5[tsv_read5.lang=="en"]


tsv_read6 = pd.read_csv("hydrate-clean_dataset_527_clean.tsv", sep='\t')
tsv_read6.columns = column_names
tsv_read6 = tsv_read6[tsv_read6.lang=="en"]

tsv_read7 = pd.read_csv("hydrate-clean_dataset_528_clean.tsv", sep='\t')
tsv_read7.columns = column_names
tsv_read7 = tsv_read7[tsv_read7.lang=="en"]

tsv_read8 = pd.read_csv("hydrate-clean_dataset_529_clean.tsv", sep='\t')
tsv_read8.columns = column_names
tsv_read8 = tsv_read8[tsv_read8.lang=="en"]

tsv_read9 = pd.read_csv("hydrate-clean_dataset_530_clean.tsv", sep='\t')
tsv_read9.columns = column_names
tsv_read9 = tsv_read9[tsv_read9.lang=="en"]

tsv_read10 = pd.read_csv("hydrate-clean_dataset_531_clean.tsv", sep='\t')
tsv_read10.columns = column_names
tsv_read10 = tsv_read10[tsv_read10.lang=="en"]

tsv_read = pd.concat([tsv_read1, tsv_read2,tsv_read3,tsv_read4,tsv_read5,tsv_read6,tsv_read7,tsv_read8,tsv_read9,tsv_read10])
tsv_read['timestamp']

#!rm hydrate-clean_dataset_323_clean.tsv

In [ ]:
del tsv_read1, tsv_read2,tsv_read3,tsv_read4,tsv_read5,tsv_read6,tsv_read7,tsv_read8,tsv_read9,tsv_read10

In [ ]:
tsv_read['repost_id'] = ""
tsv_read = tsv_read[['message_id', 'user_id','username', "repost_id",'reply_id','message','timestamp','urls']]
tsv_read['timestamp'] = tsv_read.timestamp.apply(convert_date_to_float)
tsv_read['reply_id'] = tsv_read['reply_id'].str.replace('None','')
tsv_read.dropna(subset=['message'], inplace=True)
print (len(tsv_read))

tsv_read.to_csv("tsv_read.csv",sep=',',index=False)

4582808


In [ ]:
tsv_read = tsv_read[tsv_read.message.astype(str).str.len() > 20]

In [ ]:
tsv_read.to_csv("tsv_read.csv",sep=',',index=False)

In [ ]:
csv_filename = "tsv_read.csv"
db_name = "covidtweets_mayw45.db"

import coordination_network_toolkit as coord_net_tk
import networkx as nx

# Preprocess CSV data into database
coord_net_tk.preprocess.preprocess_csv_files(db_name, [csv_filename])

Begin preprocessing tsv_read.csv into covidtweets_mayw45.db
Done preprocessing tsv_read.csv into covidtweets_mayw45.db


In [ ]:
!compute_networks covidtweets_mayw45.db compute co_tweet --time_window 60 --output_file covidtweets_mayw45_cotweet_60s.graphml --output_format graphml

Calculating a co_tweet network on covidtweets_mayw45.db with the following settings:
    time_window: 60 seconds
    min_edge_weight: 2 co-ocurring messages
    n_cpus: 2 processors
    output_file: covidtweets_mayw45_cotweet_60s.graphml
Ensuring the necessary index exists
Namespace(command='compute', database='covidtweets_mayw45.db', include_self_loops=False, include_symmetric_edges=False, min_document_size_similarity=1, min_edge_weight=2, n_cpus=2, n_messages=10, network_type='co_tweet', output_file='covidtweets_mayw45_cotweet_60s.graphml', output_format='graphml', resolved=False, similarity_threshold=0.8, time_window=60)


In [ ]:
!compute_networks covidtweets_mayw45.db compute co_link --time_window 60 --output_file covidtweets_mayw45_colink_60s.graphml --output_format graphml

Calculating a co_link network on covidtweets_mayw45.db with the following settings:
    time_window: 60 seconds
    min_edge_weight: 2 co-ocurring messages
    n_cpus: 2 processors
    output_file: covidtweets_mayw45_colink_60s.graphml
    resolved: False
Ensuring the necessary index exists
Namespace(command='compute', database='covidtweets_mayw45.db', include_self_loops=False, include_symmetric_edges=False, min_document_size_similarity=1, min_edge_weight=2, n_cpus=2, n_messages=10, network_type='co_link', output_file='covidtweets_mayw45_colink_60s.graphml', output_format='graphml', resolved=False, similarity_threshold=0.8, time_window=60)


In [ ]:
!compute_networks covidtweets_mayw45.db compute co_reply --time_window 60 --output_file covidtweets_mayw45_coreply_60s.graphml --output_format graphml

Calculating a co_reply network on covidtweets_mayw45.db with the following settings:
    time_window: 60 seconds
    min_edge_weight: 2 co-ocurring messages
    n_cpus: 2 processors
    output_file: covidtweets_mayw45_coreply_60s.graphml
Ensuring the necessary index exists
Namespace(command='compute', database='covidtweets_mayw45.db', include_self_loops=False, include_symmetric_edges=False, min_document_size_similarity=1, min_edge_weight=2, n_cpus=2, n_messages=10, network_type='co_reply', output_file='covidtweets_mayw45_coreply_60s.graphml', output_format='graphml', resolved=False, similarity_threshold=0.8, time_window=60)


In [ ]:
!compute_networks covidtweets_mayw45.db compute co_similar_tweet --time_window 60 --output_file covidtweets_mayw45_cosimilar_60s.graphml --output_format graphml

Calculating a co_similar_tweet network on covidtweets_mayw45.db with the following settings:
    time_window: 60 seconds
    min_edge_weight: 2 co-ocurring messages
    n_cpus: 2 processors
    output_file: covidtweets_mayw45_cosimilar_60s.graphml
Tokenizing messages
